In [18]:
from dotenv import load_dotenv
load_dotenv()

from app.tasks import mail_unsub_poll
from app import api_client

In [14]:

import pandas as pd
contacts = api_client.filter_contacts({})

df = pd.DataFrame(contacts)
df["stage"] = "new"

# load env CONTACTS_CSV_PATH
from pathlib import Path
import os
CONTACTS_CSV_PATH = Path(os.environ["CONTACTS_CSV_PATH"])

#save new df
df.to_csv(CONTACTS_CSV_PATH, index=False)

In [47]:
import requests
from datetime import datetime, timezone
from typing import Iterable, Optional

CAL_BASE = "https://api.cal.com/v2"
CAL_API_VERSION = "2024-08-13"  # per docs
CAL_API_KEY = os.environ["CAL_API_KEY"]

def _iso_to_dt(s: str) -> datetime:
    # "2025-09-22T12:00:00Z" → aware UTC dt
    return datetime.fromisoformat(s.replace("Z", "+00:00")).astimezone(timezone.utc)

def cancel_all_bookings(
    reason: str = "Host-initiated cancellation",
    only_future: bool = True,
    statuses: Optional[Iterable[str]] = None,   # e.g. {"accepted","confirmed","pending"}
    cancel_subsequent_bookings: bool = True,    # for recurring (non-seated) series
    dry_run: bool = True,
    verbose: bool = True,
) -> dict:
    """
    Cancel Cal.com bookings for the authenticated user/account.

    Returns a summary dict {scanned:int, attempted:int, cancelled:int, skipped:int, errors:int}.
    """
    headers = {
        "Authorization": f"Bearer {CAL_API_KEY}",       # token must start with cal_
        "cal-api-version": CAL_API_VERSION,
        "Content-Type": "application/json",
    }

    take = 100
    skip = 0
    scanned = attempted = cancelled = skipped = errors = 0

    while True:
        # GET /v2/bookings with pagination
        r = requests.get(
            f"{CAL_BASE}/bookings",
            headers=headers,
            params={"take": take, "skip": skip},
            timeout=30,
        )
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])
        scanned += len(data)

        for bk in data:
            uid   = bk.get("uid")
            stat  = bk.get("status")  # e.g. "accepted"
            start = bk.get("start")   # ISO string

            # Skip already canceled
            if (stat or "").lower() in {"canceled", "cancelled"}:
                skipped += 1
                if verbose:
                    print(f"skip  uid={uid} status={stat}")
                continue

            # Optional status filter
            if statuses and str(stat).lower() not in {s.lower() for s in statuses}:
                skipped += 1
                if verbose:
                    print(f"skip  uid={uid} status={stat} (not in filter)")
                continue

            # Only future?
            if only_future and isinstance(start, str):
                try:
                    if _iso_to_dt(start) <= datetime.now(timezone.utc):
                        skipped += 1
                        if verbose:
                            print(f"skip  uid={uid} start={start} (past)")
                        continue
                except Exception:
                    pass  # if we can't parse, fall through and attempt

            attempted += 1
            if dry_run:
                if verbose:
                    print(f"DRY   uid={uid} would cancel (status={stat}, start={start})")
                continue

            # POST /v2/bookings/{uid}/cancel
            body = {
                "cancellationReason": reason,
                "cancelSubsequentBookings": bool(cancel_subsequent_bookings),
            }
            try:
                cr = requests.post(
                    f"{CAL_BASE}/bookings/{uid}/cancel",
                    headers=headers,
                    json=body,
                    timeout=30,
                )
                if cr.status_code // 100 == 2:
                    cancelled += 1
                    if verbose:
                        print(f"OK    uid={uid} cancelled")
                else:
                    errors += 1
                    if verbose:
                        print(f"ERR   uid={uid} {cr.status_code} {cr.text}")
            except Exception as e:
                errors += 1
                if verbose:
                    print(f"ERR   uid={uid} exception: {e}")

        # Pagination
        pag = payload.get("pagination") or {}
        if not pag.get("hasNextPage"):
            break
        skip += pag.get("itemsPerPage", take)

    return {
        "scanned": scanned,
        "attempted": attempted,
        "cancelled": cancelled,
        "skipped": skipped,
        "errors": errors,
        "dry_run": dry_run,
    }

#---- Usage examples ----
#1) See what *would* be canceled (safe first step):
summary = cancel_all_bookings()
print(summary)
#2) Actually cancel *future* accepted/confirmed/pending bookings:



skip  uid=jvfCZr4debgmXMUHXprkG9 status=cancelled
skip  uid=ocSXvzwYmC7VGapb1MNe3w status=cancelled
skip  uid=kaWHhsXuwTjceqFu8mfeUs status=cancelled
skip  uid=iiHMqU9nfDicR7VBp3Ni3P status=cancelled
skip  uid=kXpRMgAPHktbm8SxJZHjVG status=cancelled
skip  uid=iEsCXP7QCuhzxAdEhHd6Yh status=cancelled
skip  uid=dvr2jbbxfL9itDYAtFN4YP status=cancelled
skip  uid=ffcY61Nfv9NthhLFX3figm status=cancelled
skip  uid=hSv3zuad75hVDxzoM6pWwU status=cancelled
skip  uid=iVrUMmPR6aQSbwJEMnkEQB status=cancelled
skip  uid=hk8SjGR6JchFKhtccJDbPG status=cancelled
skip  uid=vE1dGf92ZGUs68nApUHSM4 status=cancelled
skip  uid=a7tvC38U6vDSs6ARze3CxF status=cancelled
skip  uid=p14KmiTMdZU4yEdNSXPyRj status=cancelled
skip  uid=8xtqbzgWDPpp4St2R2rbFL status=cancelled
skip  uid=gJWFJd9EYB7TVBJn4mkvyG status=cancelled
skip  uid=9TGrQtBpzx8o1vH3oT2oJe status=cancelled
skip  uid=33RMh4x7iFSxY63Ksa5CZw status=cancelled
skip  uid=5KbLk1bGsXBnc65svpacHe status=cancelled
skip  uid=36enXysMJkeK7doWnnKjVt status=cancelled


In [46]:
summary = cancel_all_bookings(
    "cal_live",
    reason="Schedule reset",
    only_future=True,
    statuses={},
    cancel_subsequent_bookings=True,
    dry_run=False,
)
print(summary)

skip  uid=jvfCZr4debgmXMUHXprkG9 status=cancelled
skip  uid=ocSXvzwYmC7VGapb1MNe3w status=cancelled
skip  uid=kaWHhsXuwTjceqFu8mfeUs status=cancelled
skip  uid=iiHMqU9nfDicR7VBp3Ni3P status=cancelled
skip  uid=kXpRMgAPHktbm8SxJZHjVG status=cancelled
skip  uid=iEsCXP7QCuhzxAdEhHd6Yh status=cancelled
skip  uid=dvr2jbbxfL9itDYAtFN4YP status=cancelled
skip  uid=ffcY61Nfv9NthhLFX3figm status=cancelled
skip  uid=hSv3zuad75hVDxzoM6pWwU status=cancelled
skip  uid=iVrUMmPR6aQSbwJEMnkEQB status=cancelled
skip  uid=hk8SjGR6JchFKhtccJDbPG status=cancelled
skip  uid=vE1dGf92ZGUs68nApUHSM4 status=cancelled
skip  uid=a7tvC38U6vDSs6ARze3CxF status=cancelled
skip  uid=p14KmiTMdZU4yEdNSXPyRj status=cancelled
skip  uid=8xtqbzgWDPpp4St2R2rbFL status=cancelled
skip  uid=gJWFJd9EYB7TVBJn4mkvyG status=cancelled
skip  uid=9TGrQtBpzx8o1vH3oT2oJe status=cancelled
skip  uid=33RMh4x7iFSxY63Ksa5CZw status=cancelled
skip  uid=5KbLk1bGsXBnc65svpacHe status=cancelled
skip  uid=36enXysMJkeK7doWnnKjVt status=cancelled


In [ ]:
import requests
from datetime import datetime, timedelta, timezone
from typing import Dict, Any, List

CAL_BASE = "https://api.cal.com/v2"
CAL_API_VERSION = "2024-08-13"  # required

def get_bookings_created_within(
    hours: float = 1,
    *,
    take: int = 100,
    timeout_s: int = 60,
) -> List[Dict[str, Any]]:
    since_utc = datetime.now(timezone.utc) - timedelta(hours=hours)
    since_iso = since_utc.replace(microsecond=0).isoformat().replace("+00:00", "Z")

    headers = {
        "Authorization": f"Bearer {CAL_API_KEY}",
        "cal-api-version": CAL_API_VERSION,
    }

    out: List[Dict[str, Any]] = []
    skip = 0
    while True:
        params: Dict[str, str] = {
            "take": str(take),
            "afterCreatedAt": since_iso,     # server-side filter by created time
            "sortCreated": "desc",           # newest first
            "skip" : str(skip)
        }

        r = requests.get(f"{CAL_BASE}/bookings", headers=headers, params=params, timeout=timeout_s)
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])
        out.extend(data)
        pag = payload.get("pagination") or {}
        if not pag.get("hasNextPage"):
            break
        skip += pag.get("itemsPerPage", take)

    return out

bookings = get_bookings_created_within(1)

bookings


[{'id': 11093752,
  'uid': '9TGrQtBpzx8o1vH3oT2oJe',
  'title': 'VDS Discovery Project between David Vikstrand and David Vikstrand',
  'description': '',
  'hosts': [{'id': 1762985,
    'name': 'Vikstrand Deep Solutions',
    'email': 'info@vdsai.se',
    'username': 'vdsai',
    'timeZone': 'Europe/Stockholm'}],
  'status': 'cancelled',
  'cancellationReason': 'Schedule reset',
  'cancelledByEmail': None,
  'rescheduledByEmail': None,
  'start': '2025-09-30T09:00:00.000Z',
  'end': '2025-09-30T09:30:00.000Z',
  'duration': 30,
  'eventTypeId': 3349790,
  'eventType': {'id': 3349790, 'slug': 'discovery'},
  'meetingUrl': 'https://us05web.zoom.us/j/81722248941?pwd=ZVkFZOWoo496ao0iqwx02b7v3LbPg2.1',
  'location': 'https://us05web.zoom.us/j/81722248941?pwd=ZVkFZOWoo496ao0iqwx02b7v3LbPg2.1',
  'absentHost': False,
  'createdAt': '2025-09-22T13:11:42.466Z',
  'updatedAt': '2025-09-22T13:22:56.234Z',
  'metadata': {},
  'rating': None,
  'icsUid': '9TGrQtBpzx8o1vH3oT2oJe@Cal.com',
  'attende

In [45]:
bookings[0]["bookingFieldsResponses"]

{'email': 'c0nspiracy0f0nee@gmail.com',
 'name': {'firstName': 'David', 'lastName': 'Vikstrand'},
 'guests': [],
 'Company': 'VDS',
 'location': {'value': 'integrations:zoom', 'optionValue': ''}}